In [1]:
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()
my_secret = user_secrets.get_secret("wandb_api_key") 
wandb.login(key=my_secret)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
!pip install "/kaggle/input/autocorrect/autocorrect-2.6.1.tar"

Processing /kaggle/input/autocorrect/autocorrect-2.6.1.tar
  Preparing metadata (setup.py) ... - \ done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622383 sha256=d9005fba10c1ba7e16a4e70093f93513be22a98a4658590358335c725da36813
  Stored in directory: /root/.cache/pip/wheels/db/69/42/0fb0421d2fe70d195a04665edc760cfe5fd341d7bb8d8e0aaa
Successfully built autocorrect


In [3]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold, GroupKFold
from transformers import AutoTokenizer,TrainingArguments, Trainer, AutoModelForSequenceClassification,DataCollatorWithPadding, get_linear_schedule_with_warmup
# from torch.utils.data import DataLoader, Dataset
import numpy as np
import shutil
import torch.nn as nn
import json
import torch
from datasets import Dataset
from torch.optim import AdamW
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas()
from autocorrect import Speller

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
config = {
    "name":"fixed_summary",
    "random_state": 42,
    "n_folds":4,
    "model_path":"microsoft/deberta-v3-large",
    "lr":1.6e-5,
    "lr_decay": 0.95,
    "weight_decay":0.03,
    "hidden_dropout_prob":0.007,
    "attention_probs_dropout_prob":0.007,
    "n_epochs":5,
    "batch_size":2,
    "maxlen": 512,
    "strategy": "GROUPKFOLD",
    "loss":"MSELoss",
    "grad_acc_steps":2,
    "freeze_layers":["embedding","top_half"],
    "save_steps":100
}
with open("config.json", "w") as outfile:
    json.dump(config, outfile)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(config["model_path"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
train_df = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv")
train_prompts_df = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv")
train_df = train_df.merge(train_prompts_df, on="prompt_id")

speller = Speller(lang='en')
train_df["fixed_summary_text"] = train_df["text"].progress_apply(lambda x: speller(x))


train_df['input'] = train_df["prompt_question"] + "[SEP]" + train_df["fixed_summary_text"]

train_df.head()

100%|██████████| 7165/7165 [08:39<00:00, 13.80it/s]


,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text,fixed_summary_text,input
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,The third wave was an experimental see how peo...,Summarize how the Third Wave developed over su...
1,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,The Third Wave developed rapidly because the ...,Summarize how the Third Wave developed over su...
2,0095993991fe,814d6b,The third wave only started as an experiment w...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,The third wave only started as an experiment w...,Summarize how the Third Wave developed over su...
3,00c20c6ddd23,814d6b,The experimen was orginally about how even whe...,0.567975,0.969062,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,The experiment was originally about how even w...,Summarize how the Third Wave developed over su...
4,00d40ad10dc9,814d6b,The third wave developed so quickly due to the...,-0.910596,-0.081769,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,The third wave developed so quickly due to the...,Summarize how the Third Wave developed over su...


In [8]:
train_df["text_token"] = train_df["text"].progress_apply(lambda x: tokenizer(x)["input_ids"])
train_df["prompt_text_token"] = train_df["prompt_text"].progress_apply(lambda x: tokenizer(x)["input_ids"])
train_df["word_overlap_counts"] = train_df.progress_apply(lambda row: len(set(row["text_token"])&set(row["prompt_text_token"])), axis=1)

100%|██████████| 7165/7165 [00:00<00:00, 29506.76it/s]


In [9]:
train_df["text_len"] = train_df["text_token"].apply(len)
train_df["prompt_text_len"] = train_df["prompt_text_token"].apply(len)
train_df[["text_len","prompt_text_len"]].describe()

,text_len,prompt_text_len
count,7165.000000,7165.000000
mean,90.193301,809.563294
std,64.764554,205.257191
min,28.000000,651.000000
25%,48.000000,651.000000
50%,69.000000,721.000000
75%,110.000000,1137.000000
max,822.000000,1137.000000


In [10]:
train_df["len"] = train_df["input"].apply(lambda x: len(tokenizer(x)["input_ids"]))
train_df[["len"]].describe()

,len
count,7165.000000
mean,110.506629
std,64.567730
min,44.000000
25%,68.000000
50%,91.000000
75%,131.000000
max,839.000000


In [11]:
train_df.groupby(['prompt_id'])['content'].describe()

,count,mean,std,min,25%,50%,75%,max
prompt_id,,,,,,,,
39c16e,2057.0,-0.095457,0.969773,-1.729859,-0.810574,-0.157460,0.297031,3.900326
3b9047,2009.0,0.049579,1.106129,-1.729859,-0.799545,-0.093814,0.702059,3.894070
814d6b,1103.0,0.150306,1.124158,-1.729859,-0.627647,-0.093814,0.793637,3.711374
ebad26,1996.0,-0.087906,0.990271,-1.638511,-0.861984,-0.093814,0.440020,3.503226


In [12]:
train_df.groupby(['prompt_id'])['wording'].describe()

,count,mean,std,min,25%,50%,75%,max
prompt_id,,,,,,,,
39c16e,2057.0,-0.140749,1.055695,-1.962614,-0.970213,-0.168734,0.500911,4.187398
3b9047,2009.0,-0.068542,0.952708,-1.795491,-0.794551,-0.045439,0.506755,4.231226
814d6b,1103.0,0.518733,1.107806,-1.795491,-0.125597,0.503833,1.133263,4.310693
ebad26,1996.0,-0.299023,0.930270,-1.795491,-1.050372,-0.284364,0.380538,3.638126


In [13]:
def create_folds(df, n_folds, strategy):
    df["fold"] = -1
    # just use for spliting the data
    df["bins"] = pd.cut(
        df["content"], bins=n_folds, labels=False
    )
    if strategy == "KFOLD":
        kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=config["random_state"])
        for i, (train , val) in enumerate(kf.split(X=df, y=df["bins"].values)):
            df.loc[val, 'fold'] = i
            
    elif strategy == "GROUPKFOLD":
        gkf = StratifiedGroupKFold(n_splits=n_folds, shuffle=True, random_state=config["random_state"])
        for i, (train , val) in enumerate(gkf.split(X=df, y=df["bins"].values, groups=df["prompt_id"].values)):
            df.loc[val, 'fold'] = i
            
    df = df.drop("bins", axis=1)
    return df

train_df = create_folds(train_df, config["n_folds"], config["strategy"])
train_df["fold"].value_counts()

1    2057
0    2009
2    1996
3    1103
Name: fold, dtype: int64

In [14]:
def compute_metrics(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

def tokenize_function(examples):
    labels = [examples["content"], examples["wording"]]
    tokenized = tokenizer(examples["text"],
                     padding=False,
                     truncation=True,
                     max_length=config["maxlen"])
    return {
        **tokenized,
        "labels": labels,
    }

def tokenize_function_test(examples):
    tokenized = tokenizer(examples["text"],
                     padding=False,
                     truncation=True,
                     max_length=config["maxlen"])
    return tokenized

def freeze_layers(model, strategy_list):
    for s in strategy_list:
        if s == "top_half":
            for i in range(0,13,1):
                for n,p in model.deberta.encoder.layer[i].named_parameters():
                    p.requires_grad = False
        if s == "embedding":
            for n,p in model.deberta.embeddings.named_parameters():
                p.requires_grad = False
    
def create_optimizer(model, config):
    no_decay = ["bias", "LayerNorm.weight"]
    lr = config["lr"]
    # Linear layers
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if "classifier" in n],
            "weight_decay": 0.0,
            "lr": lr
        }
    ]
    # Other layers
    layers = [model.deberta.embeddings] + list(model.deberta.encoder.layer)
    layers.reverse()
    for layer in layers:
        optimizer_grouped_parameters += [
            {
                "params": [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": config["weight_decay"],
                "lr": lr
            },
            {
                "params": [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr": lr
            }
        ]
        lr *= config["lr_decay"]
        
    optimizer = AdamW(optimizer_grouped_parameters, lr=config["lr"])

In [15]:
oof_df = pd.DataFrame()
for fold in range(config["n_folds"]):
    wandb.init(project="CommonLit", name=f"{config['name']}_fold{fold}")
    wandb.log(config)
    
    tr_data = train_df[train_df['fold']!=fold].reset_index(drop=True)
    va_data = train_df[train_df['fold']==fold].reset_index(drop=True)

    train_dataset_content = Dataset.from_pandas(tr_data, preserve_index=False) 
    val_dataset_content = Dataset.from_pandas(va_data, preserve_index=False) 
    train_tokenized_datasets_content = train_dataset_content.map(tokenize_function, batched=False)
    val_tokenized_datasets_content = val_dataset_content.map(tokenize_function, batched=False)
    
    args = TrainingArguments(
        output_dir=f"/tmp/commonlit",
        per_device_train_batch_size=config["batch_size"],
        per_device_eval_batch_size=config["batch_size"],
        num_train_epochs=config["n_epochs"],
        metric_for_best_model="mcrmse",
        load_best_model_at_end=True,
        learning_rate=config["lr"],
        weight_decay=config["weight_decay"],
        gradient_accumulation_steps=config["grad_acc_steps"],
        greater_is_better=False,
        save_strategy="steps",
        evaluation_strategy="steps",
        eval_steps=config["save_steps"],
        save_steps=config["save_steps"],
        save_total_limit=1
    )
    
    model = AutoModelForSequenceClassification.from_pretrained(
        config["model_path"], 
        num_labels=2,
        problem_type="regression"
    )
    freeze_layers(model, config["freeze_layers"])
    
    optimizer = create_optimizer(model, config)

    trainer = Trainer(
        model,
        args,
        train_dataset=train_tokenized_datasets_content,
        eval_dataset=val_tokenized_datasets_content,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        optimizers=(optimizer,None)
    )
    
    trainer.train()
    shutil.rmtree(f"/tmp/commonlit")
    trainer.save_model(f"commonlit_{fold}")
    
    outputs = trainer.predict(val_tokenized_datasets_content)
    predictions = outputs.predictions
    va_data[['content_pred','wording_pred']] = predictions
    oof_df = pd.concat([oof_df, va_data])
    
    wandb.finish()

wandb: Currently logged in as: norrawee. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.5
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231007_203436-aunp2le9
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run fixed_summary_fold0
wandb: ⭐️ View project at https://wandb.ai/norrawee/CommonLit
wandb: 🚀 View run at https://wandb.ai/norrawee/CommonLit/runs/aunp2le9


  0%|          | 0/5156 [00:00<?, ?ex/s]

  0%|          | 0/2009 [00:00<?, ?ex/s]

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

Step,Training Loss,Validation Loss,Content Rmse,Wording Rmse,Mcrmse
100,No log,0.509132,0.650446,0.771481,0.710964
200,No log,0.531559,0.582749,0.850600,0.716675
300,No log,0.488884,0.621887,0.768781,0.695334
400,No log,0.649646,0.611872,0.961719,0.786795
500,0.458300,0.684433,0.615990,0.994697,0.805343
600,0.458300,0.500525,0.602585,0.798712,0.700648
700,0.458300,0.582117,0.568447,0.917117,0.742782
800,0.458300,0.537812,0.579280,0.860265,0.719773
900,0.458300,0.444173,0.566774,0.753070,0.659922
1000,0.288400,0.428457,0.513146,0.770452,0.641799


...

  0%|          | 0/6062 [00:00<?, ?ex/s]

  0%|          | 0/1103 [00:00<?, ?ex/s]

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

Step,Training Loss,Validation Loss,Content Rmse,Wording Rmse,Mcrmse
100,No log,0.818709,0.680131,1.083900,0.882016
200,No log,0.683706,0.838400,0.815168,0.826784
300,No log,1.068775,0.769328,1.243256,1.006292
400,No log,0.826342,0.793167,1.011716,0.902442
500,0.454600,0.402913,0.522865,0.729683,0.626274
600,0.454600,0.513557,0.592653,0.822117,0.707385
700,0.454600,0.594648,0.670838,0.859810,0.765324
800,0.454600,0.458658,0.603063,0.744063,0.673563
900,0.454600,0.482572,0.552884,0.812073,0.682479
1000,0.301900,0.483973,0.553476,0.813395,0.683435


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:   attention_probs_dropout_prob ▁
wandb:                     batch_size ▁
wandb:              eval/content_rmse ▅█▇▃▃▂▃▇▂▄▆▆▁▃█▆▅▅▄▅▆▆▅▆▄▅▆▅▆▇▅▅▆▅▆▆▅▅▆▆
wandb:                      eval/loss █▆█▃▃▃▃▅▂▃▄▄▁▂▅▄▄▃▃▃▄▄▃▃▂▃▄▄▄▄▃▃▃▃▄▃▃▃▃▄
wandb:                    eval/mcrmse █▆█▄▃▃▄▅▂▃▅▄▁▂▆▅▄▃▃▃▄▄▄▄▃▃▄▄▄▄▃▃▄▃▄▄▄▄▄▄
wandb:                   eval/runtime ▆▃▄▁▅▃▄▃▃▂▄▅▇▅▇▃▅▄▆▄▅▅▅▄▇▄██▅▅▄▁▄▂▃▂▅▅▄▄
wandb:        eval/samples_per_second ▃▆▅█▄▆▅▆▆▇▅▄▂▄▂▆▄▅▃▅▄▄▄▅▂▅▁▁▄▄▅█▅▇▆▇▄▄▅▅
wandb:          eval/steps_per_second ▃▆▅█▄▆▅▆▆▇▅▄▂▄▂▆▄▅▃▅▄▄▄▅▂▅▁▁▄▄▅█▅▇▆▇▄▄▅▅
wandb:              eval/wording_rmse █▄▇▄▂▄▄▃▂▂▃▂▁▁▃▃▃▁▂▁▂▁▂▁▁▂▂▃▂▁▁▂▂▂▁▁▁▁▁▁
wandb:                 grad_acc_steps ▁
wandb:            hidden_dropout_prob ▁
wandb:                             lr ▁
wandb:                       lr_decay ▁
wandb:                         maxlen ▁
wandb:                       n_epochs ▁
wandb:                        n_folds ▁
wand

In [16]:
predictions = oof_df[['content_pred','wording_pred']].values
label = oof_df[['content','wording']].values
eval_pred = predictions, label
compute_metrics(eval_pred)

{'content_rmse': 0.45558872073710593,
 'wording_rmse': 0.6050135822863296,
 'mcrmse': 0.5303011515117177}

In [17]:
oof_df.to_csv('oof_df.csv')